In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from imblearn.over_sampling import SMOTE
from collections import Counter
import torch
import torch.nn as nn
import torch.utils.data as data_utils
import torch.nn.functional as F


from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    roc_auc_score, 
    roc_curve
)



from sklearn.metrics import classification_report
from torch.utils.data import TensorDataset, DataLoader
from tqdm.notebook import tqdm



import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)


In [ ]:
df2 = pd.read_parquet(r"E:\Thesis\Defence\Datasets\CIC_IDS_2017_Binary_label_is_Label.parquet")
print(f"Dataset CIC IDS 2017 Shape: {df2.shape}")
display(df2.tail(10))

In [ ]:
plt.figure(figsize=(8, 5))
sns.countplot(x='Label', data=df2)
plt.title('Label Distribution')
plt.xlabel('Label')
plt.ylabel('count')
plt.show()

In [ ]:
x_bin2 = df2.drop(columns=['Label'])
y_bin2 = df2['Label'] 

X_train_bin, X_test_bin, y_train_bin, y_test_bin = train_test_split(
    x_bin2, y_bin2, test_size=0.2, random_state=42, stratify=y_bin2
)
print("Class distribution before:", Counter(y_train_bin))

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as data_utils
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns

class ImprovedAutoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dims=[64, 32], dropout_rate=0.1):
        super(ImprovedAutoencoder, self).__init__()
        
        
        encoder_layers = []
        decoder_layers = []
        

        prev_dim = input_dim
        for dim in encoding_dims:
            encoder_layers.extend([
                nn.Linear(prev_dim, dim),
                nn.BatchNorm1d(dim),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_dim = dim
    
        encoder_layers = encoder_layers[:-1]
        self.encoder = nn.Sequential(*encoder_layers)
        
        prev_dim = encoding_dims[-1]
        for dim in reversed(encoding_dims[:-1]):
            decoder_layers.extend([
                nn.Linear(prev_dim, dim),
                nn.BatchNorm1d(dim),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_dim = dim
        
        decoder_layers.extend([
            nn.Linear(prev_dim, input_dim),
            nn.Sigmoid()
        ])
        self.decoder = nn.Sequential(*decoder_layers)
        
        self.apply(self._init_weights)
    
    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.zeros_(m.bias)
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
    def encode(self, x):
        return self.encoder(x)

class AutoencoderEvaluator:
    def __init__(self, model, device):
        self.model = model
        self.device = device
    
    def comprehensive_evaluation(self, X_train, X_val, X_test, feature_names=None):
        """Comprehensive evaluation avoiding data leakage"""
        results = {}
        
        # 1. Reconstruction Quality
        results['reconstruction'] = self._evaluate_reconstruction(X_train, X_val, X_test)
        
        # 2. Feature Preservation (proper evaluation)
        results['feature_preservation'] = self._evaluate_feature_preservation(
            X_train, X_val, X_test, feature_names
        )
        
        # 3. Latent Space Analysis
        results['latent_analysis'] = self._analyze_latent_space(X_train, X_val, X_test)
        
        # 4. Comparison with PCA
        results['pca_comparison'] = self._compare_with_pca(X_train, X_val, X_test)
        
        return results
    
    def _evaluate_reconstruction(self, X_train, X_val, X_test):
        """Evaluate reconstruction quality on all splits"""
        self.model.eval()
        reconstruction_metrics = {}
        
        for name, X in [('train', X_train), ('val', X_val), ('test', X_test)]:
            with torch.no_grad():
                X_tensor = torch.tensor(X, dtype=torch.float32).to(self.device)
                reconstructed = self.model(X_tensor).cpu().numpy()
                
                mse = mean_squared_error(X, reconstructed)
                reconstruction_metrics[f'{name}_mse'] = mse
                reconstruction_metrics[f'{name}_rmse'] = np.sqrt(mse)
        
        return reconstruction_metrics
    
    def _evaluate_feature_preservation(self, X_train, X_val, X_test, feature_names):
        """Proper feature preservation evaluation using separate test set"""
        self.model.eval()
        
        with torch.no_grad():
            # Get latent representations
            X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(self.device)
            X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(self.device)
            
            latent_train = self.model.encode(X_train_tensor).cpu().numpy()
            latent_test = self.model.encode(X_test_tensor).cpu().numpy()
        
        # Evaluate each feature's preservation
        preservation_scores = []
        
        for i in range(X_train.shape[1]):
            # Train linear model on training data
            reg = LinearRegression()
            reg.fit(latent_train, X_train[:, i])
            
            # Evaluate on test data
            y_pred = reg.predict(latent_test)
            r2 = r2_score(X_test[:, i], y_pred)
            preservation_scores.append(max(0, r2))  # Ensure non-negative
        
        return {
            'individual_scores': preservation_scores,
            'mean_preservation': np.mean(preservation_scores),
            'median_preservation': np.median(preservation_scores),
            'features_above_90': sum(np.array(preservation_scores) > 0.9),
            'features_above_70': sum(np.array(preservation_scores) > 0.7),
            'worst_features': np.argsort(preservation_scores)[:5],
            'best_features': np.argsort(preservation_scores)[-5:]
        }
    
    def _analyze_latent_space(self, X_train, X_val, X_test):
        """Analyze properties of the latent space"""
        self.model.eval()
        
        with torch.no_grad():
            X_tensor = torch.tensor(X_train, dtype=torch.float32).to(self.device)
            latent = self.model.encode(X_tensor).cpu().numpy()
        
        return {
            'latent_dim': latent.shape[1],
            'mean_activation': np.mean(latent, axis=0),
            'std_activation': np.std(latent, axis=0),
            'correlation_matrix': np.corrcoef(latent.T),
            'effective_rank': np.linalg.matrix_rank(latent),
            'condition_number': np.linalg.cond(latent)
        }
    
    def _compare_with_pca(self, X_train, X_val, X_test):
        """Compare autoencoder performance with PCA baseline"""
        # Get autoencoder latent dimension
        with torch.no_grad():
            X_tensor = torch.tensor(X_train[:100], dtype=torch.float32).to(self.device)
            latent_dim = self.model.encode(X_tensor).shape[1]
        
        # PCA comparison
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        pca = PCA(n_components=latent_dim)
        pca.fit(X_train_scaled)
        
        # PCA reconstruction
        X_test_pca = pca.transform(X_test_scaled)
        X_test_reconstructed_pca = pca.inverse_transform(X_test_pca)
        X_test_reconstructed_pca = scaler.inverse_transform(X_test_reconstructed_pca)
        
        # Autoencoder reconstruction
        with torch.no_grad():
            X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(self.device)
            X_test_reconstructed_ae = self.model(X_test_tensor).cpu().numpy()
        
        return {
            'pca_mse': mean_squared_error(X_test, X_test_reconstructed_pca),
            'autoencoder_mse': mean_squared_error(X_test, X_test_reconstructed_ae),
            'pca_explained_variance_ratio': pca.explained_variance_ratio_,
            'pca_cumulative_variance': np.cumsum(pca.explained_variance_ratio_)
        }
    
    def plot_evaluation_results(self, results):
        """Create comprehensive visualization of results"""
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        
        # 1. Reconstruction Error Comparison
        recon = results['reconstruction']
        splits = ['train', 'val', 'test']
        mse_values = [recon[f'{split}_mse'] for split in splits]
        
        axes[0, 0].bar(splits, mse_values)
        axes[0, 0].set_title('Reconstruction MSE by Split')
        axes[0, 0].set_ylabel('MSE')
        
        # 2. Feature Preservation Distribution
        preservation = results['feature_preservation']['individual_scores']
        axes[0, 1].hist(preservation, bins=20, alpha=0.7)
        axes[0, 1].axvline(np.mean(preservation), color='red', linestyle='--', 
                          label=f'Mean: {np.mean(preservation):.3f}')
        axes[0, 1].set_title('Feature Preservation Score Distribution')
        axes[0, 1].set_xlabel('R² Score')
        axes[0, 1].legend()
        
        # 3. PCA vs Autoencoder Comparison
        pca_mse = results['pca_comparison']['pca_mse']
        ae_mse = results['pca_comparison']['autoencoder_mse']
        axes[0, 2].bar(['PCA', 'Autoencoder'], [pca_mse, ae_mse])
        axes[0, 2].set_title('PCA vs Autoencoder Reconstruction')
        axes[0, 2].set_ylabel('MSE')
        
        # 4. PCA Explained Variance
        explained_var = results['pca_comparison']['pca_cumulative_variance']
        axes[1, 0].plot(range(1, len(explained_var) + 1), explained_var, 'o-')
        axes[1, 0].set_title('PCA Cumulative Explained Variance')
        axes[1, 0].set_xlabel('Number of Components')
        axes[1, 0].set_ylabel('Cumulative Explained Variance')
        axes[1, 0].grid(True)
        
        # 5. Latent Space Correlation
        corr_matrix = results['latent_analysis']['correlation_matrix']
        im = axes[1, 1].imshow(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1)
        axes[1, 1].set_title('Latent Space Correlation Matrix')
        plt.colorbar(im, ax=axes[1, 1])
        
        # 6. Feature Preservation by Index
        axes[1, 2].plot(preservation)
        axes[1, 2].axhline(y=0.9, color='red', linestyle='--', alpha=0.7, label='90% threshold')
        axes[1, 2].axhline(y=0.7, color='orange', linestyle='--', alpha=0.7, label='70% threshold')
        axes[1, 2].set_title('Feature Preservation by Feature Index')
        axes[1, 2].set_xlabel('Feature Index')
        axes[1, 2].set_ylabel('R² Score')
        axes[1, 2].legend()
        
        plt.tight_layout()
        plt.show()
def train_and_evaluate_autoencoder(X_train, X_val, X_test, device):
    """Complete training and evaluation pipeline"""
    
    # Initialize improved model
    input_dim = X_train.shape[1]
    model = ImprovedAutoencoder(input_dim, encoding_dims=[64, 32], dropout_rate=0.1).to(device)
    
    # Training setup
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-5)
    criterion = nn.MSELoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5)
    
    # Convert to tensors
    train_tensor = torch.tensor(X_train, dtype=torch.float32)
    val_tensor = torch.tensor(X_val, dtype=torch.float32)
    
    # DataLoaders
    train_loader = data_utils.DataLoader(
        data_utils.TensorDataset(train_tensor, train_tensor),
        batch_size=512, shuffle=True
    )
    val_loader = data_utils.DataLoader(
        data_utils.TensorDataset(val_tensor, val_tensor),
        batch_size=512, shuffle=False
    )
    
    # Training loop with better practices
    best_val_loss = float('inf')
    patience_counter = 0
    train_losses, val_losses = [], []
    
    for epoch in range(100):  # Increased epochs
        # Training
        model.train()
        train_loss = 0
        for batch_x, _ in train_loader:
            batch_x = batch_x.to(device)
            
            optimizer.zero_grad()
            reconstructed = model(batch_x)
            loss = criterion(reconstructed, batch_x)
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch_x, _ in val_loader:
                batch_x = batch_x.to(device)
                reconstructed = model(batch_x)
                loss = criterion(reconstructed, batch_x)
                val_loss += loss.item()
        
        # Record losses
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        
        # Learning rate scheduling
        scheduler.step(avg_val_loss)
        
        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            # Save best model
            torch.save(model.state_dict(), r'E:\Thesis\Defence\ModelWeights\AE_CIC2017.pth')
        else:
            patience_counter += 1
            if patience_counter >= 10:
                print(f"Early stopping at epoch {epoch+1}")
                break
        
        if epoch % 10 == 0:
            print(f"Epoch {epoch+1}: Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}")
    
    # Load best model
    model.load_state_dict(torch.load(r'E:\Thesis\Defence\ModelWeights\AE_CIC2017.pth'))
    
    # Comprehensive evaluation
    evaluator = AutoencoderEvaluator(model, device)
    results = evaluator.comprehensive_evaluation(X_train, X_val, X_test)
    
    # Plot results
    evaluator.plot_evaluation_results(results)
    
    # Print summary
    print("COMPREHENSIVE AUTOENCODER EVALUATION")
    
    print(f"\nReconstruction Quality:")
    for key, value in results['reconstruction'].items():
        print(f"  {key}: {value:.6f}")
    
    print(f"\nFeature Preservation:")
    fp = results['feature_preservation']
    print(f"  Mean preservation: {fp['mean_preservation']:.4f}")
    print(f"  Median preservation: {fp['median_preservation']:.4f}")
    print(f"  Features >90% preserved: {fp['features_above_90']}")
    print(f"  Features >70% preserved: {fp['features_above_70']}")
    
    print(f"\nComparison with PCA:")
    pc = results['pca_comparison']
    print(f"  PCA MSE: {pc['pca_mse']:.6f}")
    print(f"  Autoencoder MSE: {pc['autoencoder_mse']:.6f}")
    print(f"  Improvement over PCA: {((pc['pca_mse'] - pc['autoencoder_mse']) / pc['pca_mse'] * 100):.2f}%")
    
    return model, results


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter
import torch

# Your existing data preparation
x_bin2 = df2.drop(columns=['Label'])
y_bin2 = df2['Label'] 

# First split: separate out final test set (20%)
X_temp, X_test_final, y_temp, y_test_final = train_test_split(
    x_bin2, y_bin2, test_size=0.2, random_state=42, stratify=y_bin2
)

# Second split: divide remaining 80% into train (64%) and validation (16%)
# This gives us 64% train, 16% validation, 20% test
X_train_bin, X_val_bin, y_train_bin, y_val_bin = train_test_split(
    X_temp, y_temp, test_size=0.2, random_state=42, stratify=y_temp
)

print("Dataset splits:")
print(f"Training set: {X_train_bin.shape[0]} samples ({X_train_bin.shape[0]/len(df2)*100:.1f}%)")
print(f"Validation set: {X_val_bin.shape[0]} samples ({X_val_bin.shape[0]/len(df2)*100:.1f}%)")
print(f"Test set: {X_test_final.shape[0]} samples ({X_test_final.shape[0]/len(df2)*100:.1f}%)")

print("\nClass distribution:")
print("Training:", Counter(y_train_bin))
print("Validation:", Counter(y_val_bin))
print("Test:", Counter(y_test_final))

# Convert to numpy arrays for the autoencoder
X_train = X_train_bin.values
X_val = X_val_bin.values  
X_test = X_test_final.values

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nUsing device: {device}")

# Now you can use the improved autoencoder
print("\nTraining improved autoencoder...")
model, results = train_and_evaluate_autoencoder(X_train, X_val, X_test, device)

# After training, you can extract compressed features for your classification task
print("\nExtracting compressed features for classification...")

# Get compressed representations
model.eval()
with torch.no_grad():
    X_train_compressed = model.encode(torch.tensor(X_train, dtype=torch.float32).to(device)).cpu().numpy()
    X_val_compressed = model.encode(torch.tensor(X_val, dtype=torch.float32).to(device)).cpu().numpy()
    X_test_compressed = model.encode(torch.tensor(X_test, dtype=torch.float32).to(device)).cpu().numpy()

print(f"Original feature dimensions: {X_train.shape[1]}")
print(f"Compressed feature dimensions: {X_train_compressed.shape[1]}")
print(f"Compression ratio: {X_train_compressed.shape[1]/X_train.shape[1]:.3f}")

# Optional: Save the compressed features for later use
# np.save('X_train_compressed.npy', X_train_compressed)
# np.save('X_val_compressed.npy', X_val_compressed) 
# np.save('X_test_compressed.npy', X_test_compressed)
# np.save('y_train.npy', y_train_bin.values)
# np.save('y_val.npy', y_val_bin.values)
# np.save('y_test.npy', y_test_final.values)

# print("Compressed features saved to disk.")

# Example: Quick classification test on compressed features
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

print("\nTesting classification performance on compressed features...")

# Train classifier on compressed features
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_compressed, y_train_bin)

# Evaluate
val_pred = clf.predict(X_val_compressed)
test_pred = clf.predict(X_test_compressed)

print(f"\nClassification Results on Compressed Features:")
print(f"Validation Accuracy: {accuracy_score(y_val_bin, val_pred):.4f}")
print(f"Test Accuracy: {accuracy_score(y_test_final, test_pred):.4f}")

# Compare with original features
print("\nComparing with original features...")
clf_original = RandomForestClassifier(n_estimators=100, random_state=42)
clf_original.fit(X_train, y_train_bin)

val_pred_orig = clf_original.predict(X_val)
test_pred_orig = clf_original.predict(X_test)

print(f"Original Features - Validation Accuracy: {accuracy_score(y_val_bin, val_pred_orig):.4f}")
print(f"Original Features - Test Accuracy: {accuracy_score(y_test_final, test_pred_orig):.4f}")

print(f"\nAccuracy difference (compressed vs original):")
print(f"Validation: {accuracy_score(y_val_bin, val_pred) - accuracy_score(y_val_bin, val_pred_orig):.4f}")
print(f"Test: {accuracy_score(y_test_final, test_pred) - accuracy_score(y_test_final, test_pred_orig):.4f}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn

x_bin2 = df2.drop(columns=['Label'])
y_bin2 = df2['Label'] 

# First split: separate out final test set (20%)
X_temp, X_test_final, y_temp, y_test_final = train_test_split(
    x_bin2, y_bin2, test_size=0.2, random_state=42, stratify=y_bin2
)

# Second split: divide remaining 80% into train (64%) and validation (16%)
# This gives us 64% train, 16% validation, 20% test
X_train_bin, X_val_bin, y_train_bin, y_val_bin = train_test_split(
    X_temp, y_temp, test_size=0.2, random_state=42, stratify=y_temp
)

def analyze_dataset_complexity(X_data, feature_names=None):
    """Comprehensive analysis of why your autoencoder works 'too well'"""
    
    print("="*60)
    print("DATASET COMPLEXITY ANALYSIS")
    print("="*60)
    
    # 1. Sparsity Analysis
    sparsity_ratio = np.mean(X_data == 0)
    near_zero_ratio = np.mean(np.abs(X_data) < 1e-6)
    
    print(f"\n1. SPARSITY ANALYSIS:")
    print(f"   Exact zeros: {sparsity_ratio:.1%}")
    print(f"   Near-zero values (< 1e-6): {near_zero_ratio:.1%}")
    print(f"   Effective non-zero features: {100-near_zero_ratio*100:.1f}%")
    
    # 2. Value Range Analysis
    print(f"\n2. VALUE RANGE ANALYSIS:")
    print(f"   Min value: {np.min(X_data):.2e}")
    print(f"   Max value: {np.max(X_data):.2e}")
    print(f"   Mean value: {np.mean(X_data):.2e}")
    print(f"   Std deviation: {np.std(X_data):.2e}")
    
    # Count features by magnitude
    tiny_features = np.sum(np.max(np.abs(X_data), axis=0) < 1e-5)
    small_features = np.sum((np.max(np.abs(X_data), axis=0) >= 1e-5) & 
                           (np.max(np.abs(X_data), axis=0) < 1e-2))
    normal_features = np.sum(np.max(np.abs(X_data), axis=0) >= 1e-2)
    
    print(f"   Features with max < 1e-5: {tiny_features}")
    print(f"   Features with max 1e-5 to 1e-2: {small_features}")
    print(f"   Features with max >= 1e-2: {normal_features}")
    
    # 3. Correlation Analysis
    corr_matrix = np.corrcoef(X_data.T)
    high_corr_pairs = np.sum(np.abs(corr_matrix) > 0.9) - X_data.shape[1]  # Subtract diagonal
    moderate_corr_pairs = np.sum((np.abs(corr_matrix) > 0.7) & (np.abs(corr_matrix) <= 0.9))
    
    print(f"\n3. CORRELATION ANALYSIS:")
    print(f"   Highly correlated pairs (>0.9): {high_corr_pairs//2}")
    print(f"   Moderately correlated pairs (0.7-0.9): {moderate_corr_pairs//2}")
    print(f"   Max correlation: {np.max(corr_matrix[corr_matrix < 1]):.3f}")
    
    # 4. Effective Dimensionality
    # Use PCA to understand true dimensionality
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_data)
    pca = PCA()
    pca.fit(X_scaled)
    
    # Find components that explain 95% and 99% variance
    cumvar = np.cumsum(pca.explained_variance_ratio_)
    dim_95 = np.argmax(cumvar >= 0.95) + 1
    dim_99 = np.argmax(cumvar >= 0.99) + 1
    
    print(f"\n4. EFFECTIVE DIMENSIONALITY:")
    print(f"   Original dimensions: {X_data.shape[1]}")
    print(f"   Dimensions for 95% variance: {dim_95}")
    print(f"   Dimensions for 99% variance: {dim_99}")
    print(f"   First component explains: {pca.explained_variance_ratio_[0]:.1%}")
    print(f"   First 5 components explain: {np.sum(pca.explained_variance_ratio_[:5]):.1%}")
    
    # 5. Feature Variance Analysis
    feature_vars = np.var(X_data, axis=0)
    zero_var_features = np.sum(feature_vars < 1e-10)
    low_var_features = np.sum((feature_vars >= 1e-10) & (feature_vars < 1e-6))
    
    print(f"\n5. FEATURE VARIANCE ANALYSIS:")
    print(f"   Zero/near-zero variance features: {zero_var_features}")
    print(f"   Very low variance features: {low_var_features}")
    print(f"   Informative features: {X_data.shape[1] - zero_var_features - low_var_features}")
    
    return {
        'sparsity_ratio': sparsity_ratio,
        'near_zero_ratio': near_zero_ratio,
        'effective_dims_95': dim_95,
        'effective_dims_99': dim_99,
        'high_corr_pairs': high_corr_pairs//2,
        'zero_var_features': zero_var_features,
        'pca_first_component': pca.explained_variance_ratio_[0],
        'corr_matrix': corr_matrix,
        'pca_components': pca.explained_variance_ratio_
    }

def visualize_dataset_issues(X_data, analysis_results):
    """Create visualizations showing why autoencoder works 'too well'"""
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # 1. Feature value distribution
    axes[0, 0].hist(X_data.flatten(), bins=50, alpha=0.7, log=True)
    axes[0, 0].set_title('Distribution of All Feature Values\n(Log Scale)')
    axes[0, 0].set_xlabel('Feature Value')
    axes[0, 0].set_ylabel('Frequency (log)')
    
    # 2. Feature variance distribution
    feature_vars = np.var(X_data, axis=0)
    axes[0, 1].hist(np.log10(feature_vars + 1e-12), bins=30, alpha=0.7)
    axes[0, 1].set_title('Feature Variance Distribution\n(Log10 Scale)')
    axes[0, 1].set_xlabel('Log10(Variance)')
    axes[0, 1].set_ylabel('Number of Features')
    
    # 3. Correlation matrix heatmap
    corr_matrix = analysis_results['corr_matrix']
    im = axes[0, 2].imshow(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1)
    axes[0, 2].set_title('Feature Correlation Matrix')
    plt.colorbar(im, ax=axes[0, 2])
    
    # 4. PCA explained variance
    pca_vars = analysis_results['pca_components']
    axes[1, 0].plot(range(1, min(21, len(pca_vars)+1)), pca_vars[:20], 'o-')
    axes[1, 0].set_title('PCA Explained Variance\n(First 20 Components)')
    axes[1, 0].set_xlabel('Component')
    axes[1, 0].set_ylabel('Explained Variance Ratio')
    axes[1, 0].grid(True)
    
    # 5. Cumulative explained variance
    cumvar = np.cumsum(pca_vars)
    axes[1, 1].plot(range(1, min(51, len(cumvar)+1)), cumvar[:50], 'o-')
    axes[1, 1].axhline(y=0.95, color='red', linestyle='--', label='95%')
    axes[1, 1].axhline(y=0.99, color='orange', linestyle='--', label='99%')
    axes[1, 1].set_title('Cumulative Explained Variance')
    axes[1, 1].set_xlabel('Number of Components')
    axes[1, 1].set_ylabel('Cumulative Variance')
    axes[1, 1].legend()
    axes[1, 1].grid(True)
    
    # 6. Feature magnitude distribution
    max_vals = np.max(np.abs(X_data), axis=0)
    axes[1, 2].hist(np.log10(max_vals + 1e-12), bins=30, alpha=0.7)
    axes[1, 2].set_title('Feature Maximum Value Distribution\n(Log10 Scale)')
    axes[1, 2].set_xlabel('Log10(Max Absolute Value)')
    axes[1, 2].set_ylabel('Number of Features')
    
    plt.tight_layout()
    plt.show()

def test_reconstruction_difficulty(X_data):
    """Test how easy it is to reconstruct this data"""
    
    print("\n" + "="*60)
    print("RECONSTRUCTION DIFFICULTY TEST")
    print("="*60)
    
    # Test 1: Random reconstruction
    X_random = np.random.normal(0, np.std(X_data), X_data.shape)
    random_mse = np.mean((X_data - X_random)**2)
    
    # Test 2: Mean reconstruction
    X_mean = np.full_like(X_data, np.mean(X_data, axis=0))
    mean_mse = np.mean((X_data - X_mean)**2)
    
    # Test 3: Median reconstruction
    X_median = np.full_like(X_data, np.median(X_data, axis=0))
    median_mse = np.mean((X_data - X_median)**2)
    
    # Test 4: Zero reconstruction (predicting all zeros)
    zero_mse = np.mean(X_data**2)
    
    print(f"\nBaseline Reconstruction MSE:")
    print(f"   Random noise: {random_mse:.2e}")
    print(f"   Feature means: {mean_mse:.2e}")
    print(f"   Feature medians: {median_mse:.2e}")
    print(f"   All zeros: {zero_mse:.2e}")
    
    # If your autoencoder gets much lower MSE than these baselines,
    # it might actually be learning something useful
    print(f"\nInterpretation:")
    if zero_mse < 1e-5:
        print("   ⚠️  Data is extremely sparse - even predicting zeros gives very low MSE")
    if mean_mse < 1e-5:
        print("   ⚠️  Predicting feature means gives very low MSE - data has low variance")
    
    return {
        'random_mse': random_mse,
        'mean_mse': mean_mse,
        'median_mse': median_mse,
        'zero_mse': zero_mse
    }

def why_simple_autoencoder_works(X_data):
    """Comprehensive explanation of why your simple autoencoder works"""
    
    analysis = analyze_dataset_complexity(X_data)
    baseline_mse = test_reconstruction_difficulty(X_data)
    visualize_dataset_issues(X_data, analysis)
    
    print("\n" + "="*60)
    print("WHY YOUR SIMPLE AUTOENCODER 'WORKS'")
    print("="*60)
    
    reasons = []
    
    if analysis['sparsity_ratio'] > 0.5:
        reasons.append("🔸 Extreme sparsity - most values are zero")
    
    if analysis['near_zero_ratio'] > 0.8:
        reasons.append("🔸 Most values are essentially zero (< 1e-6)")
    
    if analysis['effective_dims_95'] < analysis['effective_dims_99'] * 0.7:
        reasons.append("🔸 Very low effective dimensionality")
    
    if analysis['pca_first_component'] > 0.5:
        reasons.append("🔸 First PCA component dominates variance")
    
    if analysis['zero_var_features'] > X_data.shape[1] * 0.2:
        reasons.append("🔸 Many features have zero/near-zero variance")
    
    if baseline_mse['zero_mse'] < 1e-4:
        reasons.append("🔸 Predicting all zeros gives very low MSE")
    
    print("\nYour autoencoder works 'too well' because:")
    for reason in reasons:
        print(f"   {reason}")
    
    print(f"\n🚨 CRITICAL ISSUES:")
    print(f"   • Your data is too easy to reconstruct")
    print(f"   • Low MSE doesn't mean good feature learning")
    print(f"   • Simple linear projection might work just as well")
    print(f"   • Autoencoder might be learning trivial patterns")
    
    print(f"\n💡 RECOMMENDATIONS:")
    print(f"   1. Proper feature scaling/normalization")
    print(f"   2. Remove zero-variance features")
    print(f"   3. Compare with PCA baseline")
    print(f"   4. Test on downstream classification task")
    print(f"   5. Use more challenging evaluation metrics")
    
    return analysis, baseline_mse

# Usage with your data:
analysis, baselines = why_simple_autoencoder_works(X_train_bin.values)

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as data_utils
import os
import torch.nn.init as init
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Convert data to tensors
X_train_tensor = torch.tensor(X_train_bin.values, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test_bin.values, dtype=torch.float32).to(device)

# DataLoader
batch_size = 512
train_loader = data_utils.DataLoader(
    data_utils.TensorDataset(X_train_tensor, X_train_tensor),
    batch_size=batch_size,
    shuffle=True
)
val_loader = data_utils.DataLoader(
    data_utils.TensorDataset(X_test_tensor, X_test_tensor),
    batch_size=batch_size,
    shuffle=False
)

# Define Autoencoder with Xavier Initialization
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32)
        )
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, input_dim),
            nn.Sigmoid()
        )
        
        # Apply Xavier initialization to the layers
        self.apply(self.initialize_weights)

    def initialize_weights(self, m):
        if isinstance(m, nn.Linear):
            init.xavier_uniform_(m.weight)  # Apply Xavier initialization
            if m.bias is not None:
                nn.init.zeros_(m.bias)  # Initialize biases to zero

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Initialize model
input_dim = X_train_bin.shape[1]
autoencoder = Autoencoder(input_dim).to(device)

# Path to save/load model
model_path = "E:\Thesis\Defence\ModelWeights\AE_CIC2017.pth"

# Early stopping setup
early_stopping_patience = 5
best_val_loss = float('inf')
epochs_without_improvement = 0

# If model exists, load it
if os.path.exists(model_path):
    autoencoder.load_state_dict(torch.load(model_path, map_location=device))
    autoencoder.eval()
    print("Loaded pre-trained Autoencoder from disk.")
else:
    # Train the model
    optimizer = torch.optim.Adam(autoencoder.parameters(), lr=1e-3)
    criterion = nn.MSELoss()
    epochs = 50

    train_losses = []
    val_losses = []
    
    autoencoder.train()
    for epoch in range(epochs):
        total_train_loss = 0
        total_val_loss = 0
        
        # Training loop
        for xb, _ in train_loader:
            xb = xb.to(device)  # Ensure data is on the same device as the model
            output = autoencoder(xb)
            loss = criterion(output, xb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation loop
        autoencoder.eval()
        with torch.no_grad():
            for xb, _ in val_loader:
                xb = xb.to(device)  # Ensure data is on the same device as the model
                output = autoencoder(xb)
                loss = criterion(output, xb)
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)

        # Print loss for each epoch
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}")
        
        # Early Stopping condition
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_without_improvement = 0
            torch.save(autoencoder.state_dict(), model_path)  # Save model
            print("Model saved (improved validation loss).")
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= early_stopping_patience:
                print("Early stopping triggered.")
                break  # Stop training if no improvement in validation loss

        # Switch back to training mode
        autoencoder.train()

    # Plot Training vs Validation Loss
    plt.figure(figsize=(10, 6))
    plt.plot(range(len(train_losses)), train_losses, label="Training Loss", color='blue')
    plt.plot(range(len(val_losses)), val_losses, label="Validation Loss", color='red')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training vs Validation Loss")
    plt.legend()
    plt.show()

    # Compression analysis
    compression_ratio = 32 / input_dim
    print(f"\nCompression Analysis:")
    print(f"Original dimensions: {input_dim}")
    print(f"Compressed dimensions: 32")
    print(f"Compression ratio: {compression_ratio:.4f}")
    print(f"Storage reduction: {(1-compression_ratio)*100:.2f}%")

    # Feature importance analysis
    with torch.no_grad():
        latent_representations = autoencoder.encoder(X_train_tensor).cpu().numpy()

    feature_importance = []
    for i in range(input_dim):
        y = X_train_bin.iloc[:, i].values
        model = LinearRegression().fit(latent_representations, y)
        r2 = model.score(latent_representations, y)
        feature_importance.append(r2)

    print(f"\nFeature Preservation Analysis:")
    print(f"Average preservation score: {np.mean(feature_importance):.4f}")
    print(f"Features with >90% preservation: {sum(np.array(feature_importance) > 0.9)}")


In [ ]:
autoencoder.eval()

with torch.no_grad():
    reconstructed_test = autoencoder(X_test_tensor).cpu().numpy()

reconstruction_error = np.mean((X_test_bin.values - reconstructed_test) ** 2, axis=1)

print(f"Mean Reconstruction Error: {reconstruction_error.mean():.6f}")
print(f"Std of Reconstruction Error: {reconstruction_error.std():.6f}")
